# Cleanup Resources

Each notebook has a resource cleanup section that is designed to remove just the resources it created.

This notebook is designed to remove all resources by the type created.  By being a general cleanup utility, you can skip sections that you want to remain in use.  This notebook also has a section to delete the GCS bucket created in the `00 - Initial Setup` notebook.

**Resources**
- Vertex AI: https://googleapis.dev/python/aiplatform/latest/index.html
- BigQuery: https://googleapis.dev/python/bigquery/latest/index.html
- GCS: https://googleapis.dev/python/storage/latest/client.html

---
## Setup

Parameters:

In [1]:
PROJECT_ID = 'statmike-mlops'
REGION = 'us-central1'
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

AI Platform Client Setup:

In [2]:
from google.cloud import aiplatform
from google.cloud import aiplatform_v1beta1
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
client_options = {"api_endpoint": API_ENDPOINT}
clients = {}
aiplatform.init(project=PROJECT_ID, location=REGION)
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

---
## BigQuery
- Delete the dataset `digits`

Make a client connection to BigQuery

In [3]:
from google.cloud import bigquery
clients['bq'] = bigquery.Client(project=PROJECT_ID)

list of datasets in the project:

In [4]:
datasets = list(clients['bq'].list_datasets())

delete each dataset:

In [5]:
for ds in datasets:
    print(ds.dataset_id)
    clients['bq'].delete_dataset(ds.dataset_id,delete_contents=True,not_found_ok=True)

digits
digits_training
prediction_bq_digits_code_20210714100524_2021_07_14T06_30_50_038Z


---
## Vertex AI > Datasets
- Delete datasets

In [6]:
datasets = aiplatform.TabularDataset.list()

In [7]:
for ds in datasets:
    print(ds.resource_name)
    aiplatform.TabularDataset(dataset_name=ds.resource_name).delete()

projects/691911073727/locations/us-central1/datasets/7248306105741213696
INFO:google.cloud.aiplatform.base:Deleting TabularDataset : projects/691911073727/locations/us-central1/datasets/7248306105741213696
INFO:google.cloud.aiplatform.base:Delete TabularDataset  backing LRO: projects/691911073727/locations/us-central1/operations/6805869498671824896
INFO:google.cloud.aiplatform.base:TabularDataset deleted. . Resource name: projects/691911073727/locations/us-central1/datasets/7248306105741213696
projects/691911073727/locations/us-central1/datasets/7275327703505436672
INFO:google.cloud.aiplatform.base:Deleting TabularDataset : projects/691911073727/locations/us-central1/datasets/7275327703505436672
INFO:google.cloud.aiplatform.base:Delete TabularDataset  backing LRO: projects/691911073727/locations/us-central1/operations/3311076187832320000
INFO:google.cloud.aiplatform.base:TabularDataset deleted. . Resource name: projects/691911073727/locations/us-central1/datasets/7275327703505436672


---
## Vertex AI > Feature Store
- delete feature store including entity types and features

In [8]:
clients['fs'] = aiplatform_v1beta1.FeaturestoreServiceClient(client_options=client_options)

In [9]:
fs = clients['fs'].list_featurestores(parent=PARENT)

In [10]:
for f in fs:
    print(f.name)
    clients['fs'].delete_featurestore(request = aiplatform_v1beta1.types.DeleteFeaturestoreRequest(name=f.name,force=True))

projects/691911073727/locations/us-central1/featurestores/digits_featurestore


---
## Vertex AI > Endpoints
- Delete endpoints forcing deployed models to undeploy

In [11]:
endpoints = aiplatform.Endpoint.list()

In [12]:
for ep in endpoints:
    print(ep.resource_name)
    aiplatform.Endpoint(endpoint_name=ep.resource_name).delete(force=True)

projects/691911073727/locations/us-central1/endpoints/5623500598771974144
INFO:google.cloud.aiplatform.models:Undeploying Endpoint model: projects/691911073727/locations/us-central1/endpoints/5623500598771974144
INFO:google.cloud.aiplatform.models:Undeploy Endpoint model backing LRO: projects/691911073727/locations/us-central1/endpoints/5623500598771974144/operations/4229810511815901184
INFO:google.cloud.aiplatform.models:Endpoint model undeployed. Resource name: projects/691911073727/locations/us-central1/endpoints/5623500598771974144
INFO:google.cloud.aiplatform.base:Deleting Endpoint : projects/691911073727/locations/us-central1/endpoints/5623500598771974144
INFO:google.cloud.aiplatform.base:Delete Endpoint  backing LRO: projects/691911073727/locations/us-central1/operations/1311477953279819776
INFO:google.cloud.aiplatform.base:Endpoint deleted. . Resource name: projects/691911073727/locations/us-central1/endpoints/5623500598771974144
projects/691911073727/locations/us-central1/endp

---
## Vertex AI > Models
- Delete uploaded models

In [13]:
models = aiplatform.Model.list()

In [14]:
for model in models:
    print(model.resource_name)
    aiplatform.Model(model_name=model.resource_name).delete()

projects/691911073727/locations/us-central1/models/7751688917216133120
INFO:google.cloud.aiplatform.base:Deleting Model : projects/691911073727/locations/us-central1/models/7751688917216133120
INFO:google.cloud.aiplatform.base:Delete Model  backing LRO: projects/691911073727/locations/us-central1/operations/1969003498875912192
INFO:google.cloud.aiplatform.base:Model deleted. . Resource name: projects/691911073727/locations/us-central1/models/7751688917216133120
projects/691911073727/locations/us-central1/models/7467962140691791872
INFO:google.cloud.aiplatform.base:Deleting Model : projects/691911073727/locations/us-central1/models/7467962140691791872
INFO:google.cloud.aiplatform.base:Delete Model  backing LRO: projects/691911073727/locations/us-central1/operations/5094501640271036416
INFO:google.cloud.aiplatform.base:Model deleted. . Resource name: projects/691911073727/locations/us-central1/models/7467962140691791872
projects/691911073727/locations/us-central1/models/36263916585447587

---
## Vertex AI > Batch Predictions
- delete all batch prediction jobs

In [15]:
bps = aiplatform.BatchPredictionJob.list()

In [16]:
for bp in bps:
    print(bp.resource_name)
    aiplatform.BatchPredictionJob(batch_prediction_job_name=bp.resource_name).delete()

projects/691911073727/locations/us-central1/batchPredictionJobs/2210620904201781248
INFO:google.cloud.aiplatform.base:Deleting BatchPredictionJob : projects/691911073727/locations/us-central1/batchPredictionJobs/2210620904201781248
INFO:google.cloud.aiplatform.base:Delete BatchPredictionJob  backing LRO: projects/691911073727/locations/us-central1/operations/6580689517303300096
INFO:google.cloud.aiplatform.base:BatchPredictionJob deleted. . Resource name: projects/691911073727/locations/us-central1/batchPredictionJobs/2210620904201781248


---
## Vertex AI > Experiments > Tensorboard Instances
ISSUE: remove experiments first then instances.

In [27]:
clients['tb'] = aiplatform_v1beta1.TensorboardServiceClient(client_options=client_options)

In [28]:
tbs = clients['tb'].list_tensorboards(parent=PARENT)

In [33]:
for tb in tbs:
    print(tb.name)
    clients['tb'].delete_tensorboard(request = aiplatform_v1beta1.types.DeleteTensorboardRequest(name=tb.name))

projects/691911073727/locations/us-central1/tensorboards/3200124194595536896


## Vertex AI > Experiments > Experiments

In [35]:
tbes = clients['tb'].list_tensorboard_experiments(parent='projects/691911073727/locations/us-central1/tensorboards/3200124194595536896')

NotFound: 404 The Tensorboard does not exist.

In [37]:
clients['tb'].delete_tensorboard_experiment(request=aiplatform_v1beta1.types.DeleteTensorboardExperimentRequest(name='projects/691911073727/locations/us-central1/tensorboards/3200124194595536896'))

NotFound: 404 The Tensorboard does not exist.

---
## Vertex AI > Training
- delete various jobs types
- some create with aiplatform.* and other created by aiplatform.gapic.*

In [17]:
jobsA = aiplatform.CustomTrainingJob.list()
jobsA

 resource name: projects/691911073727/locations/us-central1/trainingPipelines/7144314295986159616]

In [18]:
jobsB = aiplatform.CustomPythonPackageTrainingJob.list()
jobsB

 resource name: projects/691911073727/locations/us-central1/trainingPipelines/7144314295986159616]

In [19]:
jobsC = aiplatform.AutoMLTabularTrainingJob.list()
jobsC

 resource name: projects/691911073727/locations/us-central1/trainingPipelines/7482084268038946816,
 resource name: projects/691911073727/locations/us-central1/trainingPipelines/524022843751530496]

## Vertex AI > Training > Custom Job
- Delete custom training jobs

In [20]:
clients['job'] = aiplatform.gapic.JobServiceClient(client_options=client_options)

In [21]:
for x in clients['job'].list_custom_jobs(parent=PARENT):
    print(x.display_name)
    clients['job'].delete_custom_job(name=x.name)

05_AIP_DIGITS_20210714165425
06_AIP_DIGITS_20210714132709-custom-job
05_AIP_DIGITS_20210714131833


## Vertex AI > Training > Training Pipeline (Python Package)
- Delete training pipelines

In [22]:
clients['pipeline'] = aiplatform.gapic.PipelineServiceClient(client_options=client_options)

In [23]:
for x in clients['pipeline'].list_training_pipelines(parent=PARENT):
    print(x.display_name)
    clients['pipeline'].delete_training_pipeline(name=x.name)

equifax _2021714142137
06_AIP_DIGITS_20210714132709
bq_digits_code_20210714100524


---
## Storage Bucket Removal

- delete contents and storage bucket `statmike-mlops` (named after project)

In [24]:
from google.cloud import storage
clients['gcs'] = storage.Client()

In [25]:
bucket = clients['gcs'].get_bucket(PROJECT_ID)

In [26]:
bucket.delete(force=True)